In [6]:
string = "aaaabbbbcababaaaa"
def manualsearch(str, k):
    i = k
    while i < len(str) and str[i] == str[i-k]:
        i+=1
    return i-k
manualsearch(string,4)

0

In [7]:
def z_algorithm(string, verbose=False):
    n = len(string) 
    l =0
    r = 0
    z_values = [0]*n
    z_values[0] = n
    for k in range(1,n):
        #case 1
        if k > r:
            z= manualsearch(string, k)
            if z>0:
                l=k
                r=k+z-1
        elif r >=k:
            mirror = k-l 
            z_mirror= z_values[mirror]
            
            current_zbox_max = r- k +1
            if verbose:print(f'current k={k}, mirror index={mirror}, mirror z box:{z_mirror}, current z box max: {current_zbox_max}')


            if z_mirror< current_zbox_max:
                z = z_mirror
            elif z_mirror == current_zbox_max:
                additional_match= manualsearch(string,r)
                if verbose: print(f'additional match:{additional_match}')
                if additional_match >0:
                    z = z_mirror + additional_match-1
                else:
                    z =z_mirror
                l = k
                r = k+z -1
            elif z_mirror > current_zbox_max:
                z = r -k+1
        if verbose: print(f' current k={k} left value:{l}, right most:{r}')
        z_values[k] =z

    return z_values

    


In [204]:
stra= "abcdabcda"
z_algorithm(stra), fundamental_preprocess(stra[::-1])


([9, 0, 0, 0, 5, 0, 0, 0, 1], [9, 0, 0, 0, 5, 0, 0, 0, 1])

In [9]:
def match_length(S: str, idx1: int, idx2: int) -> int:
    """Return the length of the match of the substrings of S beginning at idx1 and idx2."""
    if idx1 == idx2:
        return len(S) - idx1
    match_count = 0
    while idx1 < len(S) and idx2 < len(S) and S[idx1] == S[idx2]:
        match_count += 1
        idx1 += 1
        idx2 += 1
    return match_count
def fundamental_preprocess(S: str) -> list[int]:
    """Return Z, the Fundamental Preprocessing of S.

    Z[i] is the length of the substring beginning at i which is also a prefix of S.
    This pre-processing is done in O(n) time, where n is the length of S.
    """
    if len(S) == 0:  # Handles case of empty string
        return []
    if len(S) == 1:  # Handles case of single-character string
        return [1]
    z = [0 for x in S]
    z[0] = len(S)
    z[1] = match_length(S, 0, 1)
    for i in range(2, 1 + z[1]):  # Optimization from exercise 1-5
        z[i] = z[1] - i + 1
    # Defines lower and upper limits of z-box
    l = 0
    r = 0
    for i in range(2 + z[1], len(S)):
        if i <= r:  # i falls within existing z-box
            k = i - l
            b = z[k]
            a = r - i + 1
            if b < a:  # b ends within existing z-box
                z[i] = b
            else:  # b ends at or after the end of the z-box, we need to do an explicit match to the right of the z-box
                z[i] = a + match_length(S, a, r + 1)
                l = i
                r = i + z[i] - 1
        else:  # i does not reside within existing z-box
            z[i] = match_length(S, 0, i)
            if z[i] > 0:
                l = i
                r = i + z[i] - 1
    return z
def good_suffix_table(S: str) -> list[int]:
    """
    Generates L for S, an array used in the implementation of the strong good-suffix rule.
    L[i] = k, the largest position in S such that S[i:] (the suffix of S starting at i) matches
    a suffix of S[:k] (a substring in S ending at k). Used in Boyer-Moore, L gives an amount to
    shift P relative to T such that no instances of P in T are skipped and a suffix of P[:L[i]]
    matches the substring of T matched by a suffix of P in the previous match attempt.
    Specifically, if the mismatch took place at position i-1 in P, the shift magnitude is given
    by the equation len(P) - L[i]. In the case that L[i] = -1, the full shift table is used.
    Since only proper suffixes matter, L[0] = -1.
    """
    L = [-1 for c in S]
    N = fundamental_preprocess(S[::-1])  # S[::-1] reverses S
    N.reverse()
    for j in range(0, len(S) - 1):
        i = len(S) - N[j]
        if i != len(S):
            L[i] = j
    return L

def full_shift_table(S: str) -> list[int]:
    """
    Generates F for S, an array used in a special case of the good-suffix rule in the Boyer-Moore
    string search algorithm. F[i] is the length of the longest suffix of S[i:] that is also a
    prefix of S. In the cases it is used, the shift magnitude of the pattern P relative to the
    text T is len(P) - F[i] for a mismatch occurring at i-1.
    """
    F = [0 for c in S]
    Z = fundamental_preprocess(S)
    print(Z)
    longest = 0
    for i, zv in enumerate(reversed(Z)):
        longest = max(zv, longest) if zv == i + 1 else longest
        F[-i - 1] = longest
    return F
p = "acababacaba"
full_shift_table(p)


[11, 0, 1, 0, 1, 0, 5, 0, 1, 0, 1]


[11, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1]

In [161]:
stra= "accbabacaba"
# z_algorithm(str, verbose=True),fundamental_preprocess(str)
good_suffix_table(stra), good_suffix_pre(stra)

([-1, -1, -1, -1, -1, -1, -1, -1, 6, 4, 8],
 [-1, -1, -1, -1, -1, -1, -1, -1, 6, 4, 8, 9])

In [159]:
def bad_char_table(S):
    ALPHABET_SIZE = 26
    R = [[-1] for a in range(ALPHABET_SIZE)]
    alpha = [-1 for a in range(ALPHABET_SIZE)]
    for i, c in enumerate(S):
        
        index = ord(c) - ord('a')

        alpha[index] = i
        for j, a in enumerate(alpha):
            print(j,a)
            R[j].append(a)
    return R


In [3]:
def bad_char_pre(p, verbose=False):
    """
    Complexity O(M * n_char)
    bad_char_matrix[i] = latest occurence of char i in p[0,i)
    """
    n_char =26
    bad_char_matrix = [[None] for _ in range(len(p))]
    latest_bad_char = [-1] * n_char
    for i in range(1,len(p)):
        index = ord(p[i]) - ord('a')
        latest_bad_char[index] = i
        if verbose: print(latest_bad_char)
        bad_char_matrix[i] = latest_bad_char.copy()
    return bad_char_matrix


def good_suffix_pre(p, verbose = False):
    """
    find the right most of prefix with different preceding value
    start from the end of the pattern
    """
    reversed_p = p[::-1]
    
    z_values_reversed =z_algorithm(reversed_p)
    if verbose: print(f'z suffix values{z_values_reversed} for {reversed_p}')
    z_values = z_values_reversed[::-1]
    if verbose: print(z_values)
    good_suffix = [-1] * (len(p)+1)
    for i in range(len(p)-1):
        good_suf_val= len(p) - z_values[i]
        if verbose: print(f'z value: {z_values[i]}, good suffix index is {good_suf_val}')
        good_suffix[good_suf_val] = i
        if verbose:print(f'good suffix array{good_suffix}')
    return good_suffix

def matched_prefix_pre(p, verbose = False):
    """
    use z algorithm  z suffix go left to right , mp value fill right to left
    
    """
    mp =[0] * (len(p)+1)
    reversed_p = p[::-1]
    z_values_reversed =z_algorithm(reversed_p)[::-1]
    max_pre =0
    for i in range(len(p)-1,-1,-1):
        start_index = len(p)-1 -i
        if verbose:print(f'start index {start_index} z value {z_values_reversed[start_index]}')
        max_pre =max(z_values_reversed[start_index], max_pre)
        mp[i] = max_pre
    return mp
    

In [5]:
def boyer_moore(str, pat, verbose = False):
    """
    preprocessing time complexity: O(pattern)
    Complexity: O(n + m)
    bad_char = matrix of [length(str)][char]
    good_suffix = [length(str)+1]
    matched_prefix = [length(str)+1]
    """
    bad_char = bad_char_pre(pat)
    if verbose: print(bad_char, len(bad_char))
    good_suffix = good_suffix_pre(pat)
    if verbose: print(f'good suffix array{good_suffix}', len(good_suffix))
    matched_prefix = matched_prefix_pre(pat)
    if verbose: print(f'matched prefix array {matched_prefix}', len(matched_prefix))
    m=len(pat)
    s_i=len(pat)-1
    p_i=len(pat)-1
    skip_region_start =0
    skip_region_end =0
    matched_index =[]
    while s_i <len(str):
        bad_char_shift =1
        good_suffix_shift =1
        if verbose:print(f'current string index: {s_i}, current pattern index: {p_i}')
        if str[s_i] == pat[p_i]:
            #check skip region
            #to do only skip if its not bad char
            s_i-=1
            p_i-=1
            if p_i == skip_region_start:
                p_i = skip_region_end-1
                s_i -= (skip_region_start - skip_region_end+1)
                skip_region_start =0
                skip_region_end =0
        else:
            skip_region_start =0
            skip_region_end =0
            #missmatch
            if verbose:print(f'mismatched char: {str[s_i], pat[p_i]} at index {s_i}')
            
            mismatched_char = ord(str[s_i]) -ord('a')
            bad_char_index =bad_char[p_i][mismatched_char]
            if bad_char_index == -1:
                bad_char_shift = m-p_i
            else:
                bad_char_shift = p_i - bad_char_index

            if verbose: print(f'bad char shift: {bad_char_shift}')

            #good suffix. select the index where it matches
            good_suffix_index = good_suffix[p_i+1]
            print(good_suffix_index)
            #no good suffix
            if(good_suffix_index >0 and p_i <m-1):
                good_suffix_shift = m-1 - good_suffix_index
                skip_region_start = good_suffix_index
                skip_region_end = good_suffix_index - (m -2- p_i)
                if verbose: print(f'pattern index: {p_i}good suffix shift: {good_suffix_shift}, good suffix length: {m-1 -p_i} skip region: {skip_region_start} to {skip_region_end}')

            elif (p_i < m-1):
                good_suffix_shift = m-1 -matched_prefix[p_i]+1
                skip_region_start = matched_prefix[p_i]
                skip_region_end = 0
                if verbose: print(f'pattern index: {p_i} matched prefix: {matched_prefix[p_i+1]}, skip region {skip_region_start} to {skip_region_end}')
            if verbose: print(f'bad char shift: {bad_char_shift} good suffix shift: {good_suffix_shift}')

            distance_to_shift = max(bad_char_shift, good_suffix_shift)
            
            #move s_i 
            s_i += (m-1-p_i) + distance_to_shift
            #reset p_i to the end of string
            p_i= m-1

            if verbose:print(f'shift by {distance_to_shift}, new index: {s_i} compare with {str[s_i-m+1:s_i+1]}')

        if p_i ==-1:
            #fully matched
            matched_index.append(s_i+1)
            prev_matched_end = s_i + m
            p_i = len(pat)-1
            s_i+= matched_prefix[1] + m-1
            if s_i == prev_matched_end:
                s_i+=m-1
            if verbose: print(f'fully matched, new index: {s_i}')
    return matched_index
good_suffix_pre("abab")
z_algorithm("baba")

[4, 0, 2, 0]

In [215]:
# s = "acxcbbacababacabaaaabacabaafacababacaba"
# p = "acababacaba"
s= "a" * 5 + "b" * 5
p = "b" * 4
boyer_moore(s, p, verbose=True)

[[None], [-1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, 3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]] 4
good suffix array[-1, 2, 1, 0, -1] 5
matched prefix array [4, 3, 2, 1, 0] 5
current string index: 3, current pattern index: 3
mismatched char: ('a', 'b') at index 3
bad char shift: 1
-1
bad char shift: 1 good suffix shift: 1
shift by 1, new index: 4 compare with aaaa
current string index: 4, current pattern index: 3
mismatched char: ('a', 'b') at index 4
bad char shift: 1
-1
bad char shift: 1 good suffix shift: 1
shift by 1, new index: 5 compare with aaab
current string index: 5, current pattern index: 3
current string index: 4, current pattern index: 2
mismatched char: ('a', 'b') at index 4
bad char shift: 2
0
pattern index: 2 matched prefix: 1, skip region 2 to 0


[4, 6]

In [ ]:
s[6:6+11] ==p

In [4]:
from bitarray import bitarray
a = bitarray([True, True])
a


bitarray('11')

In [3]:
def bitarray_pre(pat, no_char):
    bitarray_matrix = [bitarray(0) for _ in range(no_char)]
    for character in range(no_char):
        matches = [chr(character+97) != char for char in pat]
        bitarray_matrix[character] = bitarray(matches)
    return bitarray_matrix    


In [23]:
pat = "xadad"
bitarray_pre(pat, 26)

[bitarray('10101'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11010'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('11111'),
 bitarray('01111'),
 bitarray('11111'),
 bitarray('11111')]

In [150]:
a= bitarray('110100') 
d = bitarray('11010')
x = bitarray('01111')
a[0]

1

In [35]:
x | (a << 1) | (d<<2) | (a<<3) | (d<<4)


bitarray('01111')

In [130]:
from bitarray.util import ba2int

def bitwisepm(string, pat, min_alpha="a", max_alpha= "z",verbose= False):
    matched_index= []
    alpha_range = ord(max_alpha) - ord(min_alpha) +1
    m = len(pat)
    bit_matrix = bitarray_pre(pat[::-1], alpha_range)
    bit_vector= bitarray('0' * m)
    is_matched = lambda x: ba2int(x>> m-1) ==0
    for i in range(m-1, -1, -1):
        char_index = ord(string[i]) - ord(min_alpha)
        bit_vector |= bit_matrix[char_index] << (m-1-i)
    if is_matched(bit_vector):
        matched_index.append(0)
    if verbose: print(f'bit vector 0 {bit_vector}')
    for i in range(1, len(string)-len(pat)+1):
        if verbose:print(f'string comparing {string[i:i+len(pat)]}')
        char_index = ord(string[i+m-1]) - ord('a')
        bit_vector = (bit_vector << 1) | (bit_matrix[char_index]) 
        if verbose: print(f'bit vector {i} {bit_vector}')
        if is_matched(bit_vector):
            matched_index.append(i)
    return matched_index


In [140]:
string = "abcdefg"
pat = "dadax"
bitwisepm(string, pat, verbose=True)

IndexError: string index out of range

In [148]:
a = bitarray('10101')
b =bitarray('11010')
a[4]


1

In [81]:
is_matched = lambda x: ba2int(x>> len(pat)-1) ==0

In [122]:
ord('z')

122

In [145]:
s= "mississippimississippi"
p = "issi"
s[14:14+4] == p

False